In [1]:
import extcolors

In [2]:
import PIL
img = PIL.Image.open("selenium_crawling/image/image_2.jpg")
colors, pixel_count = extcolors.extract_from_image(img)
print(colors)
print(pixel_count)

[((196, 191, 187), 496203), ((13, 13, 13), 230255), ((83, 83, 91), 8058), ((170, 52, 74), 246), ((67, 0, 1), 95), ((220, 53, 47), 35), ((36, 18, 78), 28), ((121, 127, 223), 23), ((238, 152, 155), 18), ((56, 171, 200), 16), ((153, 81, 131), 10), ((106, 71, 31), 6), ((194, 255, 216), 3), ((0, 47, 10), 3), ((28, 51, 179), 1)]
735000


In [3]:
pixel_output = 0
for c in colors:
    pixel_output += c[1]
    print(f'{c[0]} : {round((c[1] / pixel_count) * 100, 2)}% ({c[1]})')
print(f'Pixels in output: {pixel_output} of {pixel_count}')


(196, 191, 187) : 67.51% (496203)
(13, 13, 13) : 31.33% (230255)
(83, 83, 91) : 1.1% (8058)
(170, 52, 74) : 0.03% (246)
(67, 0, 1) : 0.01% (95)
(220, 53, 47) : 0.0% (35)
(36, 18, 78) : 0.0% (28)
(121, 127, 223) : 0.0% (23)
(238, 152, 155) : 0.0% (18)
(56, 171, 200) : 0.0% (16)
(153, 81, 131) : 0.0% (10)
(106, 71, 31) : 0.0% (6)
(194, 255, 216) : 0.0% (3)
(0, 47, 10) : 0.0% (3)
(28, 51, 179) : 0.0% (1)
Pixels in output: 735000 of 735000
